# Open Data Day - 2024
## Coleta de dados de recursos (em valores) disponíveis para os Objetivos de Desenvolvimento Sustentável (ODS)

Preparação dos dados para consumo no dashboard streamlit

### Fonte de dados
- url : https://brasil.un.org/pt-br/sdgs

### Atualizações
- 27/02/2024 : Início do projeto
- 28/02/2024 : Formatação do projeto

### Importação das dependências

In [ ]:
import pandas as pd
import plotly.express as px
from selenium.webdriver import ActionChains, Keys
from selenium.webdriver.common.by import By
from pathlib import Path

from config import Config
from open_data_day_2024.web_page import get_element_by_xpath, get_page_driver

### Localização do arquivos

In [ ]:
summary_file = Path.cwd() / "data" / "processed" / f"summary_recursos_disponiveis.pkl"

### Seletores XPATH

In [ ]:
COMBOBOX_ANO_XPATH = ('//h2[contains(text(), "Onde são investidos os recursos?")]'
                      '//ancestor::div[3]//label[@id="Year"]//following::div[1]')
RECURSOS_DISPONIVEIS_XPATH = '//h4[contains(text(),"Recursos Disponíveis")]'
ITEM_ODS_DIST_RECURSOS_XPATH = '//ul[contains(@class,"list")]//strong//parent::div'

### URL da página

In [ ]:
Config.URL

### Cria uma instância do ChromeWebDriver e abre a página

In [ ]:
driver = get_page_driver(Config.URL)

### Seleciona o elemento selectbox na DOM e move até a posição dele na página
Document Object Model (DOM): No contexto da programação web, DOM refere-se ao Document Object Model, que é uma interface de programação para documentos HTML, XML e SVG. Ele representa a estrutura do documento como uma árvore de objetos, onde cada nó representa parte do documento, como elementos HTML, atributos e texto.

In [ ]:
selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
ActionChains(driver).move_to_element(selectbox_ano).perform()
selectbox_ano.text

### Lista todo os anos encontrados na selectbox

In [ ]:
selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
ActionChains(driver).click(selectbox_ano).perform()
lista_anos_selectbox = driver.find_elements(By.XPATH, "//li[contains(text(), '20')]")
ANOS_SELECTBOX = list(set([ano.text for ano in lista_anos_selectbox if ano.text]))
lista_anos_selectbox[-1].send_keys(Keys.ESCAPE)
print(ANOS_SELECTBOX)

### Altera o ano no elemento selectbox e imprime o ano selecionado

In [ ]:
import time
ano = 2024

while True:
    selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
    ActionChains(driver).move_to_element(selectbox_ano).perform()
    if selectbox_ano.text == str(ano):
        break
    ActionChains(driver).click(selectbox_ano).perform()
    
    item_to_select = get_element_by_xpath(driver, f"//li[contains(text(), {ano})]")
    ActionChains(driver).click(item_to_select).perform()
    
    selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
    try:                
        if selectbox_ano.text == str(ano):
            break   
    except AttributeError:
        if item_to_select is not None:
            item_to_select.send_keys(Keys.ESCAPE)
    
selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
ActionChains(driver).move_to_element(selectbox_ano).perform()
selectbox_ano.text

### Seleciona o elemento "recursos diponíveis" e exibe o seu conteúdo

In [ ]:
recursos_disponiveis = get_element_by_xpath(driver, RECURSOS_DISPONIVEIS_XPATH)
recursos_disponiveis.text

### Seleciona a lista de itens do OSD com suas respectivas informações

In [ ]:
item_ods_recurso_list = driver.find_elements(By.XPATH, ITEM_ODS_DIST_RECURSOS_XPATH)
distribuicao_recursos = list()
for item in item_ods_recurso_list:
    titulo, percentual = item.text.splitlines()
    percentual = float(percentual.split('%')[0].replace(',', '.'))
    distribuicao_recursos.append([titulo, percentual])
    print(f'{titulo:.<50}', percentual)



### Fecha o navegador

In [ ]:
driver.close()

## Coleta e Manipulação de dados

In [ ]:
# Função que centraliza todas as ações realizadas acima
def get_dados_ods(ano: int):
    """Recupera os dados de recursos disponíveis para Os ODS com base no ano

    Args:
        ano (int): ano dos recursos disponíveis

    Returns:
        Dict[str, Any]: retorna os dados raspados inclusos em um dicionário
    """
    
    driver = get_page_driver(Config.URL)

    while True:
        selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
        ActionChains(driver).move_to_element(selectbox_ano).perform()
        if selectbox_ano.text == str(ano):
            break
        ActionChains(driver).click(selectbox_ano).perform()
        
        item_to_select = get_element_by_xpath(driver, f"//li[contains(text(), {ano})]")
        ActionChains(driver).click(item_to_select).perform()
        
        selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
        try:                
            if selectbox_ano.text == str(ano):
                break   
        except AttributeError:
            if item_to_select is not None:
                item_to_select.send_keys(Keys.ESCAPE)

    recursos_disponiveis = get_element_by_xpath(driver, RECURSOS_DISPONIVEIS_XPATH)
    recursos_disponiveis = recursos_disponiveis.text
    recursos_valores, recurso_notacao = recursos_disponiveis.split('$')[-1].split()
    recursos_valores = recursos_valores.replace(',', '.')

    if 'billion' in recurso_notacao:
        recursos_valores = float(recursos_valores) * 1_000_000_000
    elif 'million' in recurso_notacao:
        recursos_valores = float(recursos_valores) * 1_000_000
    elif 'thousand' in recurso_notacao:
        recursos_valores = float(recursos_valores) * 1_000

    dados_dist_recursos = {
        'ano': ano,
        'recursos_disponiveis': recursos_disponiveis,
        'recursos_valores': recursos_valores,
    }

    if recursos_disponiveis is not None:
        item_ods_recurso_list = driver.find_elements(By.XPATH, ITEM_ODS_DIST_RECURSOS_XPATH)
        distribuicao_recursos = list()        
        for item in item_ods_recurso_list:
            titulo, percentual = item.text.splitlines()
            percentual = float(percentual.split('%')[0].replace(',', '.'))
            distribuicao_recursos.append([titulo, percentual])
        dados_dist_recursos.update({'distribuicao_recursos': distribuicao_recursos})

    return dados_dist_recursos

get_dados_ods(2024)

### Converte os dados em DataFrames pandas

In [ ]:
df_dict = {}
for ano in sorted(ANOS_SELECTBOX):
    print(ano)

    # busca os dados dos anos existentes na selectbox dos recursos disponíveis
    dados = get_dados_ods(ano)
    
    # converte em DataFrames os dados coletados
    df = pd.DataFrame(dados['distribuicao_recursos'], columns=['Objetivo', 'Percentual'])
    df['Ano'] = dados['ano']
    df['Valor'] = df['Percentual']
    df['Valor'] = df['Valor'].apply(lambda x: (x / 100) * dados['recursos_valores'])

    # adiciona os DataFrames e seus respectivos dados no dicionário
    df_dict[ano] = (dados, df)

### Itera sobre o dicionário e exibe o conteúdo dos Dataframes

In [ ]:
for ano, registro in df_dict.items():
    dados, df = registro
    print(ano, '-', dados['recursos_disponiveis'])
    display(df)

### Concatena todos os dataframes em apenas um

In [ ]:
df = pd.concat([v[1] for k, v in df_dict.items()], ignore_index=True)
df

### Salve o arquivo de saida dentro do diretório processed

Salve o arquivo no diretório processed que esta propriamente limpo. Ele será lido e usado mais tarde para outras análises.
Outras opções além do pickle incluem:
- feather: é um formato de arquivo binário para armazenamento eficiente de dados em data frames. É projetado para   interoperabilidade entre linguagens de programação, sendo especialmente eficaz para dados tabulares. Feather é rápido para leitura e gravação, e é compatível com R e Python, tornando-o útil para transferir dados entre essas duas 

- msgpack: é um formato de serialização de dados binários que é mais eficiente em termos de espaço e tempo de processamento do que o formato JSON. Ele é projetado para ser rápido e eficiente em termos de espaço de armazenamento, e pode ser utilizado para serializar uma ampla variedade de tipos de dados, tornando-o útil para comunicação de dados entre sistemas distribuídos.

- parquet: é um formato de arquivo colunar projetado para armazenar dados tabulares de forma eficiente em termos de espaço e tempo de acesso. Ele é especialmente útil para armazenar e analisar grandes conjuntos de dados, pois permite a leitura eficiente de subconjuntos de colunas, o que pode acelerar a análise de consultas. Parquet é comumente usado em ambientes de big data, como Hadoop e Apache Spark.

In [ ]:
df.to_pickle(summary_file)